## Start Janusgraph with Cassandra backend

In [1]:
from humemai.janusgraph.utils.docker import (
    start_containers,
    stop_containers,
    remove_containers,
    remove_all_data,
    add_dummy_data,
)

start_containers(cassandra_container_name="foo", janusgraph_container_name="bar")

DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/11" 200 834
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.47/networks/janusgraph-net HTTP/11" 200 443
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.47/containers/foo/json HTTP/11" 404 37
DEBUG:janusgraph-docker:foo container not found, creating and starting it...
DEBUG:urllib3.connectionpool:http://localhost:None "POST /v1.47/containers/create?name=foo HTTP/11" 201 88
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.47/containers/e7f571b8e468e604c75c48d91073d7bcb399591f95722ee53944c6efb5dba659/json HTTP/11" 200 None
DEBUG:urllib3.connectionpool:http://localhost:None "POST /v1.47/containers/e7f571b8

## 

## Stop the two docker containers

In [4]:
stop_containers()

DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:docker.utils.config:Trying paths: ['/home/tk/.docker/config.json', '/home/tk/.dockercfg']
DEBUG:docker.utils.config:No config file found
DEBUG:urllib3.connectionpool:http://localhost:None "GET /version HTTP/11" 200 834
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.47/containers/cassandra/json HTTP/11" 200 None
DEBUG:urllib3.connectionpool:http://localhost:None "POST /v1.47/containers/cac5db93a8bd41e45456435e13f6c9b360f2eebdc99b6d1a639566f0d757ac5f/stop HTTP/11" 304 0
DEBUG:janusgraph-docker:cassandra container stopped.
DEBUG:urllib3.connectionpool:http://localhost:None "GET /v1.47/containers/janusgraph/json HTTP/11" 200 None
DEBUG:urllib3.connectionpool:http://localhost:None "POST /v1.47/containers/eedb15a1175d4df90ba9b73494de2390c13cec7711402e4615b97adc2e2d0eca/stop HTTP/11" 304 0
DEBUG:janusgraph-docker:janusgraph container

## Run a Gremlin query to remove all the data

In [3]:
remove_all_data()

INFO:gremlinpython:Creating DriverRemoteConnection with url 'ws://localhost:8182/gremlin'
INFO:gremlinpython:Creating Client with url 'ws://localhost:8182/gremlin'
INFO:gremlinpython:Creating GraphTraversalSource.
INFO:gremlinpython:Creating GraphTraversalSource.
DEBUG:gremlinpython:submit with bytecode '[['V'], ['drop'], ['none']]'
DEBUG:gremlinpython:message '[['V'], ['drop'], ['none']]'
DEBUG:gremlinpython:processor='traversal', op='bytecode', args='{'gremlin': [['V'], ['drop'], ['none']], 'aliases': {'g': 'g'}}'
DEBUG:asyncio:Using selector: EpollSelector
DEBUG:janusgraph-docker:All vertices and edges have been deleted.
INFO:gremlinpython:closing DriverRemoteConnection with url 'ws://localhost:8182/gremlin'
INFO:gremlinpython:Closing Client with url 'ws://localhost:8182/gremlin'


## ADd dummy data

In [2]:
add_dummy_data()

INFO:gremlinpython:Creating DriverRemoteConnection with url 'ws://localhost:8182/gremlin'
INFO:gremlinpython:Creating Client with url 'ws://localhost:8182/gremlin'
INFO:gremlinpython:Creating GraphTraversalSource.
INFO:gremlinpython:Creating GraphTraversalSource.
DEBUG:gremlinpython:submit with bytecode '[['addV', 'person'], ['property', 'name', 'Alice'], ['property', 'age', 30]]'
DEBUG:gremlinpython:message '[['addV', 'person'], ['property', 'name', 'Alice'], ['property', 'age', 30]]'
DEBUG:gremlinpython:processor='traversal', op='bytecode', args='{'gremlin': [['addV', 'person'], ['property', 'name', 'Alice'], ['property', 'age', 30]], 'aliases': {'g': 'g'}}'
DEBUG:asyncio:Using selector: EpollSelector
DEBUG:gremlinpython:submit with bytecode '[['addV', 'person'], ['property', 'name', 'Bob'], ['property', 'age', 25]]'
DEBUG:gremlinpython:message '[['addV', 'person'], ['property', 'name', 'Bob'], ['property', 'age', 25]]'
DEBUG:gremlinpython:processor='traversal', op='bytecode', args='

In [12]:
import nest_asyncio
import asyncio
from gremlin_python.driver.serializer import GraphSONSerializersV3d0
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.structure.graph import Graph

# Apply nest_asyncio to allow nested event loops (useful in Jupyter notebooks)
nest_asyncio.apply()

# Set up the connection using GraphSON 3.0
graph = Graph()
connection = DriverRemoteConnection(
    'ws://localhost:8182/gremlin',
    'g',
    message_serializer=GraphSONSerializersV3d0()
)
g = graph.traversal().withRemote(connection)


try:
    # Query 1: Retrieve all vertices with their properties
    vertices = g.V().elementMap().toList()
    print("Vertices:")
    for vertex in vertices:
        print(vertex)

    # Query 2: Retrieve all edges with their properties
    edges = g.E().elementMap().toList()
    print("\nEdges:")
    for edge in edges:
        print(edge)

finally:
    # Close the connection
    connection.close()

Vertices:
{'name': 'Carol', 'age': 27, <T.id: 1>: 4096, <T.label: 4>: 'person'}
{'name': 'Alice', 'age': 30, <T.id: 1>: 4104, <T.label: 4>: 'person'}
{'name': 'Dave', 'age': 35, <T.id: 1>: 8200, <T.label: 4>: 'person'}
{'name': 'Acme Corp', 'type': 'Company', <T.id: 1>: 4152, <T.label: 4>: 'organization'}
{'name': 'Bob', 'age': 25, <T.id: 1>: 4272, <T.label: 4>: 'person'}
{'name': 'Globex Inc', 'type': 'Company', <T.id: 1>: 40964320, <T.label: 4>: 'organization'}

Edges:
{<T.id: 1>: {'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'odyww-35s-4r9-6bs'}}, <T.label: 4>: 'knows', <Direction.IN: 'IN'>: {<T.id: 1>: 8200, <T.label: 4>: 'person'}, <Direction.OUT: 'OUT'>: {<T.id: 1>: 4096, <T.label: 4>: 'person'}, 'since': 2019}
{<T.id: 1>: {'@type': 'janusgraph:RelationIdentifier', '@value': {'relationId': 'odzb4-35s-6c5-37c'}}, <T.label: 4>: 'works_at', <Direction.IN: 'IN'>: {<T.id: 1>: 4152, <T.label: 4>: 'organization'}, <Direction.OUT: 'OUT'>: {<T.id: 1>: 4096, <T.label:

In [17]:
vertices

[{'name': 'Carol', 'age': 27, <T.id: 1>: 4096, <T.label: 4>: 'person'},
 {'name': 'Alice', 'age': 30, <T.id: 1>: 4104, <T.label: 4>: 'person'},
 {'name': 'Dave', 'age': 35, <T.id: 1>: 8200, <T.label: 4>: 'person'},
 {'name': 'Acme Corp',
  'type': 'Company',
  <T.id: 1>: 4152,
  <T.label: 4>: 'organization'},
 {'name': 'Bob', 'age': 25, <T.id: 1>: 4272, <T.label: 4>: 'person'},
 {'name': 'Globex Inc',
  'type': 'Company',
  <T.id: 1>: 40964320,
  <T.label: 4>: 'organization'}]